encoder transformer层的linear1层（etl1）替换为近似矩阵乘法

In [1]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories

In [2]:
# root_path = '/data/hdr/csi_transformer/pic/'
# print(root_path)
# root_path2 = '/'.join(root_path.split("/")[:-2])
# # sys.path.append(root_path)
# print(root_path2)

In [3]:
method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
# method = METHOD_SCALAR_QUANTIZE

In [4]:
linear_name = 'etl2'
feedback_bits = 256
ncodebooks = 512 # max:512
ncentroids = 256
if method == METHOD_MITHRAL:
    ncentroids = 16
train_sam_num = 50 # 训练集样本数
test_sam_num = 1000 # 测试集样本数
batch_size = 32

In [5]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= ''
    featurepath_train= ''
    data_to_fcpath_test = ''
    featurepath_test = ''
elif host_name == 'jm-System-Product-Name':
    dir_train = os.path.join('/data/hdr/transformer_data/joined', 'train', 'f'+str(feedback_bits))
    dir_test = os.path.join('/data/hdr/transformer_data/joined', 'test', 'f'+str(feedback_bits))
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'ex_linear2in_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'ex_linear2_y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'ex_linear2out_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'ex_linear2in_test.npy'
    featurepath_test = 'ex_linear2out_test.npy'
    # y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'ex_linear2_w_f256.npy'
biaspath = 'ex_linear2_b_f256.npy'
dir_result = os.path.join(dir_result, method, "f%i" % feedback_bits, linear_name)
try:
    os.mkdir(dir_result)
except FileExistsError:
    pass 


In [6]:
est3 = mm.estFactory(X_path=data_to_fcpath_train, W_path=weightpath, Y_path=y_train, dir= dir_train, ncodebooks=ncodebooks, ncentroids=ncentroids, methods=[method])


running method:  Mithral
_learn_mithral_initialization heuristic pq
learn_multisplits(): initial loss:    7.183620929718018
learn_multisplits(): returning loss:  0.02316792931742384
learn_multisplits(): initial loss:    1.7999025583267212
learn_multisplits(): returning loss:  0.004802439591003349
learn_multisplits(): initial loss:    1.0954980850219727
learn_multisplits(): returning loss:  2.4915350422816118e-05
learn_multisplits(): initial loss:    8.47905445098877
learn_multisplits(): returning loss:  0.016014074351915042
learn_multisplits(): initial loss:    0.4075913727283478
learn_multisplits(): returning loss:  0.0010110686724971174
learn_multisplits(): initial loss:    100.24736785888672
learn_multisplits(): returning loss:  0.37082407996058464
learn_multisplits(): initial loss:    5911.62109375
learn_multisplits(): returning loss:  25.29250192642212
learn_multisplits(): initial loss:    0.02854217030107975
learn_multisplits(): returning loss:  0
learn_multisplits(): initial los

/data/hdr/pq/bolt/experiments/python/clusterize.py:783: RuntimeWarning: divide by zero encountered in double_scalars
  scale = 254. / upper_val


learn_multisplits(): initial loss:    5450.103515625
learn_multisplits(): returning loss:  22.207971334457397
learn_multisplits(): initial loss:    37.017608642578125
learn_multisplits(): returning loss:  0.19271257385844365
learn_multisplits(): initial loss:    1.249909520149231
learn_multisplits(): returning loss:  0.0014240499804145657
learn_multisplits(): initial loss:    67.55850219726562
learn_multisplits(): returning loss:  1.2937266454100609
learn_multisplits(): initial loss:    234.5479736328125
learn_multisplits(): returning loss:  3.1853117742575705
learn_multisplits(): initial loss:    18.321269989013672
learn_multisplits(): returning loss:  0.06079623629921116
learn_multisplits(): initial loss:    17.46610450744629
learn_multisplits(): returning loss:  0.047358763880765764
learn_multisplits(): initial loss:    0.432903915643692
learn_multisplits(): returning loss:  0.00020867178682237864
learn_multisplits(): initial loss:    0.0003716962819453329
learn_multisplits(): retur

In [7]:
# type(est3)

In [8]:
x_test = np.load(dir_test+'/'+data_to_fcpath_test)
w_test = np.load(dir_train+'/'+weightpath)
bias = np.load(dir_train+'/'+biaspath)
# print(type(est3))
y_out_matmul = mm.eval_matmul(est3, x_test, w_test) # MADDNESS乘法的结果
# y_out_last = mu.softmax(y_out_matmul + bias.T) # MADDNESS替换后当前层输出，即+bias并激活函数后的结果
y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

In [9]:
print(y_out_last)
print("y_out_last.shape: ", y_out_last.shape)
y_out_last_re = y_out_last.reshape(test_sam_num, batch_size, -1, y_out_last.shape[-1])
print("y_out_last_re.shape: ", y_out_last_re.shape)
# np.save("LDPC_decoder_NET_testdata/" + snr + "nomul_matmul_yout_matmul", y_out_matmul)
# np.save(dir_result+'/'+method+'fc1_fb256_cb%i_ct%i.npy' % (ncodebooks, ncentroids), y_out_matmul)
np.save(os.path.join(dir_result, '%s%s_trsam%i_tesam%i_fb%i_cb%i_ct%i.npy' % (method, linear_name, train_sam_num, test_sam_num, feedback_bits, ncodebooks, ncentroids)), y_out_last_re)
# io.savemat(dir_result+'\\fc1_256.mat', {"NN_output_buffer": y_out_last})

[[ 0.20397455  0.42263836  0.4235347  ... -0.06638381 -0.24724624
  -0.11866097]
 [ 0.20397455  0.42263836  0.4235347  ... -0.06638381 -0.24724624
  -0.3691502 ]
 [ 0.20397455  0.42263836  0.4235347  ... -0.06638381 -0.24724624
  -0.3691502 ]
 ...
 [ 0.20397455  0.17214912 -0.07744379 ... -0.31687304  0.25373223
  -0.11866097]
 [-0.0465147  -0.07834011 -0.327933   ... -0.06638381  0.5042215
  -0.3691502 ]
 [-0.0465147  -0.07834011  0.17304546 ... -0.06638381 -0.7482247
  -0.11866097]]
y_out_last.shape:  (1024000, 64)
y_out_last_re.shape:  (1000, 32, 32, 64)
